In [1]:
print("ok")

ok


In [2]:
%pwd

'/home/ajay-yadav/MedicalChatBot/End-to-End-medical-chatbot/research'

In [3]:
import os
os.chdir("../")

In [4]:
%pwd

'/home/ajay-yadav/MedicalChatBot/End-to-End-medical-chatbot'

In [5]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [6]:
#Extract Data from PDF Files
def load_pdf_file(data) :
    loader = DirectoryLoader(data, glob="*.pdf", loader_cls=PyPDFLoader)
    documents = loader.load()
    print(f'You have {len(documents)} documents in your data')
    return documents

In [7]:
%pwd


'/home/ajay-yadav/MedicalChatBot/End-to-End-medical-chatbot'

In [8]:
import os
os.listdir()

['requirements.txt',
 'Data',
 'research',
 'Generative_AI_Project.egg-info',
 'LICENSE',
 '.env',
 'template.py',
 '.gitignore',
 'setup.py',
 'README.md',
 'src',
 '.git',
 'app.py']

In [9]:
extracted_data = load_pdf_file(data="Data/")

You have 755 documents in your data


In [11]:
# extracted_data

In [12]:
#split the data into text chunks

def text_split(extracted_data) :
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)
    print(f'Now you have {len(text_chunks)} chunks of data')
    return text_chunks

In [13]:
text_chunks = text_split(extracted_data)

Now you have 6733 chunks of data


In [14]:
from langchain.embeddings import HuggingFaceEmbeddings

In [15]:
#Download the embedding model from HuggingFace

def download_hugging_face_embeddings() :
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [16]:
embeddings = download_hugging_face_embeddings()

/tmp/ipykernel_178921/135544804.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/home/ajay-yadav/miniconda3/envs/medibot/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [17]:
query_result = embeddings.embed_query("What is COVID-19?")
print("length",len(query_result)) 

length 384


In [18]:
from dotenv import load_dotenv
load_dotenv()

True

In [19]:
PINECONE_API_KEY = os.environ.get("PINECONE_API_KEY")


In [20]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)
index_name = "medical-chatbot"
pc.create_index(name=index_name,
                dimension=384, 
                metric="cosine", 
                spec=ServerlessSpec(
                    cloud="aws", region="us-east-1")
        )

{
    "name": "medical-chatbot",
    "metric": "cosine",
    "host": "medical-chatbot-kywhcku.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [21]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY


In [22]:
#Embed each chunk and upsert the embeddings to Pinecone
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(text_chunks, embeddings, index_name=index_name)

/home/ajay-yadav/miniconda3/envs/medibot/lib/python3.10/site-packages/langchain_pinecone/__init__.py:3: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_pinecone.vectorstores import Pinecone, PineconeVectorStore


In [23]:
#Load existing index
from langchain_pinecone import PineconeVectorStore
docsearch = PineconeVectorStore.from_existing_index(embedding=embeddings,index_name=index_name)

In [24]:
docsearch

In [25]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [26]:
retrieved_docs = retriever.invoke("What is COVID-19?")

In [27]:
retrieved_docs

[Document(id='0559d423-2997-4049-b4eb-5f66846493b8', metadata={'author': 'giac', 'creationdate': '2005-06-26T21:37:13+02:00', 'creator': 'PScript5.dll Version 5.2.2', 'moddate': '2005-07-08T10:51:49+02:00', 'page': 318.0, 'page_label': '319', 'producer': 'Acrobat Distiller 7.0 (Windows)', 'source': 'Data/test and pathological analysis.pdf', 'title': 'Fischbach - A Manual of Laborat', 'total_pages': 755.0}, page_content='shock, multiorgan failure, and lung edema. Jaundice occurs in yellow fever. Heavy rains are associated with an increase \nin number of rodents (which are the vectors in HF) and mosquitoes in yellow fever. The hantavirus, called sin nombre (no \nname), is responsible for hantavirus pulmonary syndrome (HPS). See microbiology references for more information.\nReference Values \nNormal No evidence of Hantaan virus, Ebola virus, or 17 other viruses that may cause HF in rodents or deer mice No'),
 Document(id='7edb08a1-c58f-4841-a7da-2cd040d09ce0', metadata={'author': 'giac',

In [28]:
GEMENIE_API_KEY = os.environ.get("GEMENIE_API_KEY")

In [29]:
os.environ["GEMENIE_API_KEY"] = GEMENIE_API_KEY

In [30]:
!pip install langchain-google-genai


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [31]:
!pip show langchain-google-genai
!pip list | grep langchain



huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Name: langchain-google-genai
Version: 2.1.10
Summary: An integration package connecting Google's genai package and LangChain
Home-page: https://github.com/langchain-ai/langchain-google
Author: 
Author-email: 
License: MIT
Location: /home/ajay-yadav/miniconda3/envs/medibot/lib/python3.10/site-packages
Requires: filetype, google-ai-generativelanguage, langchain-core, pydantic
Required-by: 


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


langchain                    0.3.27
langchain-community          0.3.29
langchain-core               0.3.75
langchain-experimental       0.3.4
langchain-google-genai       2.1.10
langchain-openai             0.3.32
langchain-pinecone           0.2.12
langchain-text-splitters     0.3.11


In [40]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",   # or "gemini-1.5-pro"
    temperature=0.4,
    max_output_tokens=500,
    google_api_key=os.getenv("GEMENIE_API_KEY")  # ✅ make sure the env var is set correctly
)


In [41]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

# Updated Prompt to match the required Gemini format
prompt_template = (
    "Context: {context}\n\n"
    "Input: {input}\n\n"
    "Provide me the response from the context according to the input from the user. "
    "The response should be formal and clear, maintaining the wording style of the user's input. "
    "Ensure the response is strictly from the context, declarative, and follows the user's tone. "
    "Modify the given context accordingly."
)

# Chat Prompt (Ensuring Correct Query Structure for Gemini)
prompt = ChatPromptTemplate.from_messages(
    [
        (prompt_template),
    ]
)


In [42]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)


In [44]:
response = rag_chain.invoke({"input": "what is youtube?"})
print(response["answer"])

The provided text does not contain any information about YouTube.
